## 따릉이 데이터를 활용한 데이터 분석

- 각 날짜의 1시간 전의 기상상황을 가지고 1시간 후의 따릉이 대여수를 예측해보세요. 

## 1. 라이브러리 및 데이터
## Library & Data

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor # 랜덤 포레스트

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')

## 2. 탐색적 자료분석
## Exploratory Data Analysis (EDA)


### pd.DataFrame.head()
 - 데이터 프레임의 위에서 부터 n개 행을 보여주는 함수
 - n의 기본 값(default 값)은 5

In [3]:
train.head(3)

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0


+ id : 날짜와 시간별 id
+ hour_bef_temperature : 1시간 전 기온
+ hour_bef_precipitation : 1시간 전 비 정보, 비가 오지 않았으면 0, 비가 오면 1
+ hour_bef_windspeed : 1시간 전 풍속(평균)
+ hour_bef_humidity : 1시간 전 습도
+ hour_bef_visibility : 1시간 전 시정(視程), 시계(視界)(특정 기상 상태에 따른 가시성을 의미)
+ hour_bef_ozone : 1시간 전 오존
+ hour_bef_pm10 : 1시간 전 미세먼지(머리카락 굵기의 1/5에서 1/7 크기의 미세먼지)
+ hour_bef_pm2.5 : 1시간 전 미세먼지(머리카락 굵기의 1/20에서 1/30 크기의 미세먼지)
+ count : 시간에 따른 따릉이 대여 수 

In [4]:
test.head(3)

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
0,0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0
1,1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0
2,2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0


In [5]:
submission.head(3)

,id,count
0,0,NaN
1,1,NaN
2,2,NaN


### pd.DataFrame.tail()
 - 데이터 프레임의 아래에서 부터 n개 행을 보여주는 함수
 - n의 기본 값(default 값)은 5

In [6]:
train.tail(3)

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
1456,2176,5,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,22.0
1457,2178,21,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,216.0
1458,2179,17,21.1,0.0,3.1,47.0,1973.0,0.046,38.0,17.0,170.0


### pd.DataFrame.shape
 - 데이터 프레임의 행의 개수와 열의 개수가 저장되어 있는 속성(attribute)

In [7]:
train.shape

(1459, 11)

In [8]:
test.shape

(715, 10)

In [9]:
submission.shape

(715, 2)

### pd.DataFrame.info()
- 데이터셋의 column별 정보를 알려주는 함수
- 비어 있지 않은 값은 (non-null)은 몇개인지?
- column의 type은 무엇인지?
 - type의 종류 : int(정수), float(실수), object(문자열), 등등 (date, ...)

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1459 non-null   int64  
 1   hour                    1459 non-null   int64  
 2   hour_bef_temperature    1457 non-null   float64
 3   hour_bef_precipitation  1457 non-null   float64
 4   hour_bef_windspeed      1450 non-null   float64
 5   hour_bef_humidity       1457 non-null   float64
 6   hour_bef_visibility     1457 non-null   float64
 7   hour_bef_ozone          1383 non-null   float64
 8   hour_bef_pm10           1369 non-null   float64
 9   hour_bef_pm2.5          1342 non-null   float64
 10  count                   1459 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 125.5 KB


### pd.DataFrame.describe()
- 숫자형 (int, float) column들의 기술 통계량을 보여주는 함수

- 기술통계량이란?
 - 해당 column을 대표할 수 있는 통계값들을 의미
 
 
- 기술통계량 종류
 - count: 해당 column에서 비어 있지 않은 값의 개수
 - mean: 평균
 - std: 표준편차
 - min: 최솟값 (이상치 포함)
 - 25% (Q1): 전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 1/4번째 지점에 있는 값
 - 50% (Q2): 중앙값 (전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 2/4번째 지점에 있는 값)
 - 75% (Q3): 전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 3/4번째 지점에 있는 값
 - max: 최댓값 (이상치 포함) 
 
 
 
- 이상치: 울타리 밖에 있는 부분을 이상치라고 정의함
   - 아래쪽 울타리: $Q_1$ - $1.5 * IQR$
   - 위쪽 울타리: $Q_3$ + $1.5 * IQR$
   - $IQR$ = $Q_3 - Q_1$
 
 
<img src="https://miro.medium.com/max/10125/1*NRlqiZGQdsIyAu0KzP7LaQ.png" width="700" height="500">

In [11]:
train.describe()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
count,1459.000000,1459.000000,1457.000000,1457.000000,1450.000000,1457.000000,1457.000000,1383.000000,1369.000000,1342.000000,1459.000000
mean,1105.914325,11.493489,16.717433,0.031572,2.479034,52.231297,1405.216884,0.039149,57.168736,30.327124,108.563400
std,631.338681,6.922790,5.239150,0.174917,1.378265,20.370387,583.131708,0.019509,31.771019,14.713252,82.631733
min,3.000000,0.000000,3.100000,0.000000,0.000000,7.000000,78.000000,0.003000,9.000000,8.000000,1.000000
25%,555.500000,5.500000,12.800000,0.000000,1.400000,36.000000,879.000000,0.025500,36.000000,20.000000,37.000000
50%,1115.000000,11.000000,16.600000,0.000000,2.300000,51.000000,1577.000000,0.039000,51.000000,26.000000,96.000000
75%,1651.000000,17.500000,20.100000,0.000000,3.400000,69.000000,1994.000000,0.052000,69.000000,37.000000,150.000000
max,2179.000000,23.000000,30.000000,1.000000,8.000000,99.000000,2000.000000,0.125000,269.000000,90.000000,431.000000


### pd.DataFrame.groupby()
 - 집단에 대한 통계량 확인 
 
<img src="https://s3.amazonaws.com/files.dezyre.com/images/Tutorials/Split+Apply+Combine+Strategy.png" width="700" height="500">

In [12]:
train[['hour','count']].groupby('hour').mean()

,count
hour,
0,71.766667
1,47.606557
2,31.409836
3,21.377049
4,13.524590
5,13.114754
6,24.557377
7,62.360656
8,136.688525


### plt.plot()의 스타일

색깔

|문자열|약자|
|----|-----|
|blue|b|
|green|g|
|red|r|
|cyan|c|
|magenta|m|
|yellow|y|
|black|k|
|white|w|

마커

|마커|의미|
|----|----|
|.|점|
|o|원|
|v|역삼각형|
|^|삼각형|
|s|사각형|
|*|별|
|x|엑스|
|d|다이아몬드|

선

|문자열|의미|
|-----|-----|
| - | 실선|
|-- | 끊어진 실선|
| -.| 점+실선|
|:|점선|

In [13]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot('hour', 'count', '*', data=train)

In [ ]:
train.columns

In [ ]:
plt.plot('hour','hour_bef_humidity', 'yo', data=train)

In [ ]:
plt.plot('hour', 'count', 'r+', data=train)
plt.plot('hour', 'hour_bef_visibility', 'c^', data=train) # c(cyan) : 청록색

### plt.title(label, fontsize)
- 그래프 제목 생성

### plt.xlabel(label, fontsize)
- x축 이름 설정

### plt.ylabel(label, fontsize)
- y축 이름 설정

### plt.axvline(x, color)
- 축을 가로지르는 세로 선 생성

### plt.text(x, y, s, fontsize)
- 원하는 위치에 텍스트 생성

In [ ]:
plt.title('Plot 1', fontsize=15)
plt.plot('hour', 'count', 'o', data=train) # plt.plot에 데이터 입력
plt.xlabel('hour', fontsize=12)
plt.ylabel('count', fontsize=12)
plt.axvline(8, color='red') # axvline : 수직선 추가
plt.text(3, 200, 'go to work', fontsize=10) # 텍스트 추가
plt.savefig('plot1.png') # 그림 저장

### 상관계수

- 상관계수: 두 개의 변수가 같이 일어나는 강도를 나타내는 수치 
- -1에서 1사이의 값을 지닙니다. 
- -1이나 1인 수치는 현실 세계에서 관측되기 힘든 수치입니다. 
- 분야별로 기준을 정하는 것에 따라 달라지겠지만, 보통 0.4이상이면 두 개의 변수간에 상관성이 있다고 얘기합니다. 

![상관계수](https://t1.daumcdn.net/cfile/tistory/99DEE1425C6A9F2008)

- 상관관계는 인과관계와 다릅니다. 아래의 예시를 확인해 봅시다.

![상관성 예시](https://miro.medium.com/max/684/1*JLYI5eCVEN7ZUWXBIrrapw.png)

- 선글라스 판매량이 증가함에 따라, 아이스크림 판매액도 같이 증가하는 것을 볼 수 있습니다. 
- 하지만 선글라스 판매량이 증가했기 **때문에** 아이스크림 판매액이 증가했다라고 해석하는 것은 타당하지 않습니다. 
- 선글라스 판매량이 증가했다는 것은 여름 때문이라고 볼 수 있으므로, 날씨가 더워짐에 따라 선글라스 판매량과 아이스크림 판매액이 같이 증가했다고 보는 것이 타당할 것입니다. 

### pd.DataFrame.corr()

- correlation coefficient 의 줄임말 입니다 

In [ ]:
train.corr()

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize = (12,12))
sns.heatmap(train.corr(), annot=True) # annotation : 주석

### sns.lmplot()

In [ ]:
import seaborn as sns

In [ ]:
sns.lmplot(x='hour', y='count', data=train)

### sns.kdeplot()
Kernel Density Estimation : 커널 밀도 추정

In [ ]:
train['count'].plot(kind='hist', bins=30) 

In [ ]:
sns.kdeplot(train['count'])

### sns.boxplot()
* 최대, 최소, 평균, 사분위수를 보기 위한 그래프
* 특이치를 발견하기에도 좋다
* 단일 연속형 변수에 대해 수치를 표시하거나, 연속형 변수를 기반으로 서로 다른 범주형 변수를 분석할 수 있다

In [ ]:
sns.boxplot(x='hour', y='hour_bef_humidity', data=train)

### sns.pairplot()
* 데이터 셋을 통째로 넣으면 숫자형 특성에 대하여 각각에 대한 히스토그램과 두 변수 사이의 scatter plot을 그린다

In [ ]:
sns.pairplot(train[['hour','hour_bef_humidity','hour_bef_visibility','count','hour_bef_ozone']])

### sns.jointplot()
* 두 변수에 대한 displot의 조합이다
* 두 변수의 분포에 대한 분석을 할 수 있다
* 두 displot 사이에 scatter plot이 추가되어 분포를 추가로 확인할 수 있다
* scatter plot 대신 hex plot으로 정의할 수도 있다

In [ ]:
sns.jointplot('hour', 'count', data=train, alpha=0.1)

### sns.violinplot()
* Box plot과 비슷하지만 분포에 대한 보충 정보가 제공된다

In [ ]:
sns.violinplot('hour','hour_bef_humidity',data=train)

### sns.relplot()

In [ ]:
train[train['hour'] > 12]

In [ ]:
sns.relplot(x='hour', y='hour_bef_humidity', hue = 'hour_bef_precipitation', size='count', data = train[train['hour']>12])

## 3. 데이터 전처리
## Data Cleansing & Pre-Processing  

### pd.Series.isna()
- 결측치 여부를 확인해줍니다.
- 결측치면 True, 아니면 False

In [ ]:
train.isna().sum()

### pd.DataFrame.fillna()
- 결측치를 채우고자 하는 column과 결측치를 대신하여 넣고자 하는 값을 명시해주어야 합니다.

In [ ]:
train['hour_bef_temperature'] = train['hour_bef_temperature'].fillna(value = train['hour_bef_temperature'].mean())
train.isna().sum()

In [ ]:
train_isna_sum = train.isna().sum()

In [ ]:
train_isna_sum[train_isna_sum != 0].index

In [ ]:
na_columns = train_isna_sum[train_isna_sum != 0].index

In [ ]:
def fill_bicycle_na(df, column):
    df[column] = df[column].fillna(value=df[column].mean())

In [ ]:
fill_bicycle_na(train, 'hour_bef_precipitation')

In [ ]:
for col in na_columns:
    fill_bicycle_na(train, col)
    print(col, '결측값 대체 완료')

In [ ]:
test_isna_sum = test.isna().sum()

In [ ]:
test_na_columns = test_isna_sum[test_isna_sum != 0].index

In [ ]:
for col in test_na_columns:
    print(col)

In [ ]:
for col in test_na_columns:
    fill_bicycle_na(test, col)
    print(col, '대체 완료')

In [ ]:
test.isna().sum()

## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling  

### sklearn.neighbors.KNeighborsRegressor()
- KNN 모형

![](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1531424125/KNN_final1_ibdm8a.png)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_jobs = -1)

In [ ]:
column = ['hour', 'hour_bef_temperature']
X_train, y_train, X_test = train[column], train['count'], test[column]

In [ ]:
model_5 = KNeighborsRegressor(n_jobs=-1, n_neighbors=5)
model_7 = KNeighborsRegressor(n_jobs=-1, n_neighbors=7)
model_9 = KNeighborsRegressor(n_jobs=-1, n_neighbors=9)

In [ ]:
kfold = KFold(n_splits = 5, shuffle = True, random_state = 10)

In [ ]:
np.mean(cross_val_score(model_5, X_train, y_train, cv = kfold, scoring = 'neg_mean_squared_error'))

In [ ]:
np.mean(cross_val_score(model_7, X_train, y_train, cv = kfold, scoring = 'neg_mean_squared_error'))

In [ ]:
np.mean(cross_val_score(model_9, X_train, y_train, cv = kfold, scoring = 'neg_mean_squared_error'))

In [ ]:
model_9.fit(X_train, y_train)

In [ ]:
submission['count'] = model_9.predict(X_test)

In [ ]:
submission.to_csv('knn_9.csv', index=False)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
submission['count'] = model.predict(X_test)

In [ ]:
submission.to_csv('knn_5.csv', index=False)

In [ ]:
train

### sklearn.ensemble.RandomForestRegressor()
- 랜덤 포레스트 모형

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2020/02/rfc_vs_dt1.png)

In [4]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

In [5]:
train

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0
...,...,...,...,...,...,...,...,...,...,...,...
1454,2174,4,16.8,0.0,1.6,53.0,2000.0,0.031,37.0,27.0,21.0
1455,2175,3,10.8,0.0,3.8,45.0,2000.0,0.039,34.0,19.0,20.0
1456,2176,5,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,22.0
1457,2178,21,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,216.0


In [6]:
X_train = train.drop(['id','count'], axis=1)
y_train = train['count']
X_test = test.drop('id', axis=1)

In [9]:
X_train.shape, y_train.shape, X_test.shape

((1459, 9), (1459,), (715, 9))

In [10]:
from sklearn.model_selection import GridSearchCV

In [11]:
RandomForestRegressor()

RandomForestRegressor()

In [12]:
param = {'min_samples_split' : [30, 50, 70],
         'max_depth' : [5,6,7],
         'n_estimators' : [50,150,250]}

In [13]:
param

{'min_samples_split': [30, 50, 70],
 'max_depth': [5, 6, 7],
 'n_estimators': [50, 150, 250]}

In [15]:
gs = GridSearchCV(estimator = model, param_grid = param, scoring = 'neg_mean_squared_error', cv = 3)

In [16]:
gs

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [5, 6, 7],
                         'min_samples_split': [30, 50, 70],
                         'n_estimators': [50, 150, 250]},
             scoring='neg_mean_squared_error')

In [18]:
gs.fit(X_train, y_train)

C:\Users\gmkim\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
81 fits failed out of a total of 81.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
81 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\gmkim\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\gmkim\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 327, in fit
    X, y = self._validate_data(
  File "C:\Users\gmkim\anaconda3\lib\site-packages\sklearn\base.py", line 576, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\gmkim\anaconda3\lib\sit

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [19]:
submission['count'] = gs.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

### model.fit()
- 모델 학습

### model.predict()
- 모델 예측

### pd.DataFrame.to_csv()
 - csv파일 저장하는 함수

## 교차검증 소개: Cross Validation(CV)

### 홀드아웃 교차검증기법(Hold-out Cross Validation)

![](https://www.datavedas.com/wp-content/uploads/2018/04/image003.jpg)

> `train_test_split(특징데이터, 타겟데이터, test_size= 0.25, shuffle= True, random_state= 1)`
- 특징데이터와 타겟데이터를 순서대로 넣어줍니다
- test_size: 분할할 검증용데이터 사이즈를 설정합니다(0~1)
- shuffle: 데이터를 섞습니다 <span style='color:red'> <- 순서가 중요한 데이터가 아니라면 언제나 True!</span>
- random_state: 재구현을 위해 시드값을 넣어줍니다

### K폴드 교차검증기법(K-fold Cross Validation)

![](https://www.researchgate.net/profile/Fabian_Pedregosa/publication/278826818/figure/fig10/AS:614336141750297@1523480558954/The-technique-of-KFold-cross-validation-illustrated-here-for-the-case-K-4-involves.png)

In [ ]:
from sklearn.model_selection import KFold

> `KFold(n_splits= , shuffle= , random_state= 1)`
- n_splits: 몇 폴드로 데이터를 분할할지 결정합니다
- shuffle: 데이터를 셔플할지 여부를 결정합니다
- random_state: 재현을 위해 시드값을 고정합니다

이번 검증에는 10개의 분할된 폴드를 활용한 검증을 합니다.  
다음과 같이 반복문을 이용하여 정확도를 구해봅시다.

> ``` for idx_trn, idx_val in kfolds.split(X_trn):
    train_X, valid_X = X_trn.iloc[idx_trn, :], X_trn.iloc[idx_val, :] 
    train_y, valid_y = y_trn.iloc[idx_trn], y_trn.iloc[idx_val]```
 - `k폴드.split()`함수를 통해 각 분할로 나누어진 샘플들의 <span style="color:red">인덱스</span>에 접근합니다.

## 6. 결과 및 결언
## Conclusion & Discussion

In [ ]:
!pip install scholar

In [ ]:
[공공] 서울시 따릉이 자전거 이용 예측 AI모델[공공] 서울시 따릉이 자전거 이용 예측 AI모델